In [1]:
import conf
import uuid
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()

print(role)

bucket = conf.SESSION_BUCKET
sess = sagemaker.Session(default_bucket=bucket)

arn:aws:iam::664457504912:role/service-role/AmazonSageMaker-ExecutionRole-20200423T115075


In [2]:
bucket = sess.default_bucket()
sess.default_bucket = bucket
sess.default_bucket

'hardhat-dataset-sagemaker-object-detector-solvimm'

In [3]:
from sagemaker.amazon.amazon_estimator import get_image_uri
import uuid

training_image = get_image_uri(sess.boto_region_name, 'object-detection', repo_version="latest")
print (training_image)

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


811284229777.dkr.ecr.us-east-1.amazonaws.com/object-detection:latest


In [4]:
s3_output_location = 's3://{}/output'.format(bucket)
s3_checkpoint_path = "checkpoint"
checkpoint_s3_uri =  's3://{}/{}/{}'.format(bucket, s3_checkpoint_path, uuid.uuid4())
s3_train_data = 's3://{}/{}'.format(bucket, 'train')
s3_validation_data = 's3://{}/{}'.format(bucket, 'validation')
s3_train_annotation = 's3://{}/{}'.format(bucket, 'train_annotation')
s3_validation_annotation = 's3://{}/{}'.format(bucket, 'validation_annotation')

In [5]:
od_model = sagemaker.estimator.Estimator(training_image,
                                         role, 
                                         train_instance_count=conf.train_instance_count, 
                                         train_instance_type=conf.train_instance_type,
                                         train_volume_size=conf.train_volume_size,
                                         train_use_spot_instances=conf.train_use_spot_instances,
                                         train_max_wait=conf.train_max_wait,
                                         train_max_run=conf.train_max_run,
                                         input_mode=conf.input_mode,
                                         output_path=s3_output_location,
                                         checkpoint_s3_uri=checkpoint_s3_uri,
                                         sagemaker_session=sess)

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


In [6]:
od_model.set_hyperparameters(base_network=conf.base_network,
                             use_pretrained_model=conf.use_pretrained_model,
                             num_classes=conf.num_classes,
                             mini_batch_size=conf.mini_batch_size,
                             epochs=conf.epochs,
                             learning_rate=conf.learning_rate,
                             lr_scheduler_step=conf.lr_scheduler_step,
                             lr_scheduler_factor=conf.lr_scheduler_factor,
                             optimizer=conf.optimizer,
                             momentum=conf.momentum,
                             weight_decay=conf.weight_decay,
                             overlap_threshold=conf.overlap_threshold,
                             nms_threshold=conf.nms_threshold,
                             image_shape=conf.image_shape,
                             label_width=conf.label_width,
                             num_training_samples=conf.num_training_samples)

In [7]:
train_data = sagemaker.session.s3_input(s3_train_data,
                                        distribution='FullyReplicated', 
                                        content_type='image/jpeg', 
                                        s3_data_type='S3Prefix')

validation_data = sagemaker.session.s3_input(s3_validation_data,
                                             distribution='FullyReplicated', 
                                             content_type='image/jpeg',
                                             s3_data_type='S3Prefix')

train_annotation = sagemaker.session.s3_input(s3_train_annotation,
                                              distribution='FullyReplicated', 
                                              content_type='image/jpeg',
                                              s3_data_type='S3Prefix')

validation_annotation = sagemaker.session.s3_input(s3_validation_annotation,
                                                   distribution='FullyReplicated', 
                                                   content_type='image/jpeg',
                                                   s3_data_type='S3Prefix')

data_channels = {
    'train': train_data,
    'validation': validation_data,
    'train_annotation': train_annotation,
    'validation_annotation': validation_annotation
}

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


In [ ]:
od_model.fit(inputs=data_channels, logs=True)

2020-08-11 21:55:56 Starting - Starting the training job...
2020-08-11 21:55:59 Starting - Launching requested ML instances......
2020-08-11 21:57:14 Starting - Preparing the instances for training......
2020-08-11 21:58:23 Downloading - Downloading input data............
2020-08-11 22:00:24 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
[08/11/2020 22:00:28 INFO 140448397821760] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/default-input.json: {u'label_width': u'350', u'early_stopping_min_epochs': u'10', u'epochs': u'30', u'overlap_threshold': u'0.5', u'lr_scheduler_factor': u'0.1', u'_num_kv_servers': u'auto', u'weight_decay': u'0.0005', u'mini_batch_size': u'32', u'use_pretrained_model': u'0', u'freeze_layer_pattern': u'', u'lr_scheduler_step': u'', u'early_stopping': u'False', u'early_stopping_patience': u'5', u'momentum': u'0.9', u'num_training_samples': u'', u'optimizer': 